# WG - Gesucht Data Analysis 
#### Step 1: Data Cleaning 

- Data cleaning involves filtering out premium status ads and removing duplicates. 
- Duplicates are identified by the combination of title, address, and duplicate IDs.


In [157]:
import pandas as pd
import re
import os
import os
import datetime

# Get the parent directory of the current working directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

apartmentsDataPath = os.path.join(parent_dir, 'WebCrawlerApp/data/output/apartmentsBerlinData.csv')

# Get the last modified timestamp of the CSV file
last_modified = os.path.getmtime(apartmentsDataPath)

# Convert the timestamp to a datetime object
last_modified_date = datetime.datetime.fromtimestamp(last_modified)

# Print the last modified date
print(f"The last changed date of the CSV file is: {last_modified_date.strftime('%d.%m.%Y')}")

# Read the CSV file
df = pd.read_csv(apartmentsDataPath, usecols=list(range(0, 45)))

print(f"The length of the entire dataset is: {len(df)}")

The last changed date of the CSV file is: 24.03.2024


The length of the entire dataset is: 16120


##### Step 1.1: Filter Premium Ads from the dataset

In [158]:
df_without_premium = df[df['premiumstatus'] == False]

print(f"The size of the dataset reduced by Premium ads is: {len(df_without_premium)} \n\nThere are {len(df) - len(df_without_premium)} Premium ads in the dataset.")

The size of the dataset reduced by Premium ads is: 10233 

There are 5887 Premium ads in the dataset.


##### Step 1.2: Filter duplicate IDs from the WG-Gesucht Dataset

In [159]:
# Overwrite the original DataFrame with the one without Premium ads
df = df_without_premium

def compare_duplicates (column_name: str): 
    # Step 1: Find duplicates based on apartmentID
    dup_ids = df[df.duplicated(subset=[column_name], keep=False)][column_name].unique()

    # Prepare a new DataFrame for the comparison
    comparison_columns = ['apartmentID1', 'apartmentID2', 'same_id', 'title1', 'title2', 'same_title', 'address1', 'address2', 'same_address', 'street1', 'street2', 'same_street', 'zip1', 'zip2', 'same_zip', 'city1', 'city2', 'same_city']
    df_comparison = pd.DataFrame(columns=comparison_columns)

    # Step 2: Find duplicates based on ID
    for apt_id in dup_ids:
        # Filter the DataFrame for the current apartmentID
        dup_df = df[df[column_name] == apt_id]

        # Ensure there are at least two entries for comparison
        if len(dup_df) < 2:
            print(dup_df)
            continue  
        
        # Choose the first two entries for comparison
        first, second = dup_df.iloc[0], dup_df.iloc[1]
        
        # Create a new row for the comparison DataFrame
        new_row = {
            'apartmentID1': first['apartmentID'], 
            'apartmentID2': second['apartmentID'], 
            'same_id': True,  # Da sie dieselbe apartmentID haben
            'title1': first['title'], 
            'title2': second['title'], 
            'same_title': first['title'] == second['title'], 
            'address1': first['address'], 
            'address2': second['address'], 
            'same_address': first['address'] == second['address'], 
            'street1': first['street'],
            'street2': second['street'],
            'same_street': first['street'] == second['street'],
            'zip1': first['postcode'],
            'zip2': second['postcode'],
            'same_zip': first['postcode'] == second['postcode'],
            'city1': first['city'],
            'city2': second['city'],
            'same_city': first['city'] == second['city']
        }
        df_comparison = pd.concat([df_comparison, pd.DataFrame([new_row], columns=comparison_columns)])

    print(f"Number of duplicates based on {column_name}: {len(df_comparison)}")
    return df_comparison


In [160]:
df_comparison = compare_duplicates('apartmentID')

Number of duplicates based on apartmentID: 1358


Step 1.2.1: View Same IDs and address with different titles

In [161]:
df_FalseTitle = df_comparison[(df_comparison['same_id'] == True) & (df_comparison['same_title'] == False) & (df_comparison['same_address'] == True)]

print(f"""
{len(df_FalseTitle)} duplicates based on apartmentID, that have the same address but different titles.

When you view the table, you can see that the user or owner of the add changed the title over time. 

Result: Duplicates based on ID and Address with false Title will be removed from the dataset. """)

#display_as_table(df_FalseTitle) # Here you can output the entire table of the False titles with duplicate ids and addresses


229 duplicates based on apartmentID, that have the same address but different titles.

When you view the table, you can see that the user or owner of the add changed the title over time. 

Result: Duplicates based on ID and Address with false Title will be removed from the dataset. 


Step 1.2.2: View Same IDs and Title with different address. 

In [162]:
df_FalseAddress = df_comparison[(df_comparison['same_id'] == True) & (df_comparison['same_title'] == True) & (df_comparison['same_address'] == False) ]

print(f"""
{len(df_FalseAddress)} duplicates based on apartmentID, that have the same titles but with different addresses.""")

#display_as_table(df_falseadressAndTitle)


18 duplicates based on apartmentID, that have the same titles but with different addresses.


In [163]:
from Levenshtein import ratio

def compare_streetnames(df_differentstreets):
    data_List = []

    for index, row in df_differentstreets.iterrows():
        data = {'street1' : row['street1'], 'street2' : row['street2']}
        data_List.append(data)

    more_than_80_percent_similar = 0
    between_40_and_80_percent_similar = 0
    less_than_40_percent_similar = 0

    for item in data_List:
        str1, str2 = str(item["street1"]), str(item["street2"])
        similarity = ratio(str1.lower(), str2.lower())  # Similarity as a number between 0 and 1
        if similarity > 0.8:
            more_than_80_percent_similar += 1
        elif similarity >= 0.4 and similarity <= 0.8:
            between_40_and_80_percent_similar += 1
        elif similarity < 0.4:
            less_than_40_percent_similar += 1

    print(f"""
{more_than_80_percent_similar} duplicates based on apartmentID, where the street names are more than 80% similar. --> These are probably the same street with different spellings.
{between_40_and_80_percent_similar} duplicates based on apartmentID, where the street names are between 40% and 80% similar. --> These are probably the same street with minor differences.
{less_than_40_percent_similar} duplicates based on apartmentID, where the street names are less than 40% similar. --> These are probably different streets.""")

    return less_than_40_percent_similar

In [164]:
less_than_40_percent_similar = compare_streetnames(df_FalseAddress)


14 duplicates based on apartmentID, where the street names are more than 80% similar. --> These are probably the same street with different spellings.
3 duplicates based on apartmentID, where the street names are between 40% and 80% similar. --> These are probably the same street with minor differences.
1 duplicates based on apartmentID, where the street names are less than 40% similar. --> These are probably different streets.


In [165]:
print(f""" 
The percentage of duplicates based on apartment ID, where  {((less_than_40_percent_similar / len(df_comparison)) * 100):.4f} % of the street names are less than 40% similar.
This percentage means that there may be ads that are identical in terms of apartmentID, but have different addresses and are therefore not duplicates.
Is this percentage acceptable? If it is less than 1%: {1 > ((less_than_40_percent_similar / len(df_comparison)) * 100)}
""")

if 1 > ((less_than_40_percent_similar / len(df_comparison)) * 100):
    print("The percentage is acceptable. The duplicates based on apartmentID with different addresses will be removed from the dataset.")

assert 1 > ((less_than_40_percent_similar / len(df_comparison)) * 100), "The percentage is not acceptable. The duplicates based on apartmentID with different addresses will not be removed from the dataset."

 
The percentage of duplicates based on apartment ID, where  0.0736 % of the street names are less than 40% similar.
This percentage means that there may be ads that are identical in terms of apartmentID, but have different addresses and are therefore not duplicates.
Is this percentage acceptable? If it is less than 1%: True

The percentage is acceptable. The duplicates based on apartmentID with different addresses will be removed from the dataset.


Step 1.2.3: View Same IDs with False Title and Address

In [166]:
df_AllDifferent = df_comparison[(df_comparison['same_id'] == True) & (df_comparison['same_title'] == False) & (df_comparison['same_address'] == False)]

less_than_40_percent_similar = compare_streetnames(df_AllDifferent)

#display_as_table(df_AllDifferent)


30 duplicates based on apartmentID, where the street names are more than 80% similar. --> These are probably the same street with different spellings.
8 duplicates based on apartmentID, where the street names are between 40% and 80% similar. --> These are probably the same street with minor differences.
4 duplicates based on apartmentID, where the street names are less than 40% similar. --> These are probably different streets.


In [167]:
print(f""" 
The percentage of duplicates based on apartment ID, where  {((less_than_40_percent_similar / len(df_comparison)) * 100):.4f} % of the street names are less than 40% similar.
This percentage means that there may be ads that are identical in terms of apartmentID, but have different addresses and are therefore not duplicates.
Is this percentage acceptable? If it is less than 1%: {1 > ((less_than_40_percent_similar / len(df_comparison)) * 100)}""")

assert 1 > ((less_than_40_percent_similar / len(df_comparison)) * 100), "The percentage of duplicates based on apartment ID, where the street names are less than 40% similar is too high. Please check the data."

 
The percentage of duplicates based on apartment ID, where  0.2946 % of the street names are less than 40% similar.
This percentage means that there may be ads that are identical in terms of apartmentID, but have different addresses and are therefore not duplicates.
Is this percentage acceptable? If it is less than 1%: True


In [168]:
df_AllSame = df_comparison[(df_comparison['same_id'] == True) & (df_comparison['same_title'] == True) & (df_comparison['same_address'] == True)]

print(f"""
{len(df_AllSame)} duplicates based on apartmentID, that have the same titles and the same address.
      
These are the exact duplicates and need to be removed.""")

#display_as_table(df_falseadress)


1069 duplicates based on apartmentID, that have the same titles and the same address.
      
These are the exact duplicates and need to be removed.


In [169]:
assert len(df_comparison) == (len(df_FalseTitle) + len(df_FalseAddress) + len(df_AllSame) + len(df_AllDifferent)) 

##### Step 1.3: Filter duplicates, non values, 1-room-apartments from the WG-Gesucht Dataset

In [170]:
df = df.drop_duplicates(subset='apartmentID')

print(f"The length of the dataset after removing duplicates is: {len(df)}")

df = df.dropna(subset=['room_size'])
df = df[df['room_size'] != 'n.a.']

print(f"The length of the dataset after removing NaN for roomsize is: {len(df)}")

df = df.dropna(subset=['total_rent'])

df = df.dropna(subset=['suburb'])

print(f"The length of the dataset after removing NaN for suburb is: {len(df)}")

df = df.drop_duplicates(subset=['title', 'address', 'rent'])

print(f"The length of the dataset after removing duplicates based on title and address is: {len(df)}")

df.reset_index(drop=True, inplace=True)

df_notnull = df[df['user_name'].notnull()]

df = df[df['user_name'].isnull()]

print(f"The length of the dataset after removing ads with user_name is: {len(df)}")


The length of the dataset after removing duplicates is: 8875
The length of the dataset after removing NaN for roomsize is: 8861
The length of the dataset after removing NaN for suburb is: 8717
The length of the dataset after removing duplicates based on title and address is: 8548
The length of the dataset after removing ads with user_name is: 8363


In [171]:
pattern = r'1-Zimmer-Wohnung|1-Zimmer Wohnung|1-room Apartment|1-room-apartment|1-room-studio'

df = df[~df['title'].str.contains(pattern, case=False, na=False)]

print(f"The length of the dataset after removing 1-room apartments is: {len(df)}")

The length of the dataset after removing 1-room apartments is: 8116


##### Step 1.4: Update total rent for short term contracts

In [172]:
# Clean the data and convert the columns to the right data type for calculations
df.loc[:, 'room_size'] = df['room_size'].str.replace('m²', '').astype(float)
df.loc[:, 'apartment_size'] = df['apartment_size'].str.replace('m²', '').astype(float)
df.loc[:, 'total_rent'] = df['total_rent'].str.replace('€', '').astype(float)
df.loc[:, 'max_roommate'] = df['max_roommate'].astype(float)

In [173]:
# Function to adjust rent based on availability duration
def adjust_rent(df):
    df_duration = pd.DataFrame()
    df_duration.loc[:, 'available_from'] = pd.to_datetime(df['available_from'], dayfirst=True)
    df_duration.loc[:, 'available_until'] = pd.to_datetime(df['available_until'], dayfirst=True)

    # Calculating the adjusted rent
    for index, row in df_duration.iterrows():
        if pd.notna(row['available_until']):
            diff_days = (row['available_until'] - row['available_from']).days
            if diff_days < 30:
             df.at[index, 'rent_updated'] = (df.at[index, 'total_rent'] / diff_days) * 30

    return df

In [174]:
# Inititalizing new column updated rent
df['rent_updated'] = df['total_rent'].copy()

df = adjust_rent(df)

##### Step 1.5: Clean the suburb columns for price analysis and add districts 

In [175]:
suburb_to_district_path = os.path.join(parent_dir, 'WebCrawlerApp/data/input/relationSuburbsToDistrict.xlsx')

xlsx = pd.ExcelFile(suburb_to_district_path)

suburbs_to_district_dict = {}

for sheet_name in xlsx.sheet_names:
    suburbs_to_district_dict[sheet_name] = xlsx.parse(sheet_name).iloc[:, 0].dropna().tolist()

def find_suburb_and_district(address):
    # Regex, um zu überprüfen, ob der Vorort-/Bezirksname von Leerzeichen oder Satzzeichen umgeben ist
    address = address.lower()
    for district, suburbs in suburbs_to_district_dict.items():
        for suburb in suburbs:
            # Suchmuster, das sicherstellt, dass der Name als ganzes Wort erscheint
            pattern = r'\b' + re.escape(suburb.lower()) + r'\b'
            if re.search(pattern, address):
                return suburb, district
    return None, None

df['suburb'], df['district'] = zip(*df['address'].map(find_suburb_and_district))


In [176]:
# Reorder the columns in the DataFrame
columns = list(df.columns)
total_rent_index = columns.index('total_rent')
suburb_index = columns.index('suburb')
columns_to_move = ['rent_updated', 'district']

columns.remove('rent_updated')
columns.remove('district')

columns.insert(total_rent_index + 1, 'rent_updated')
columns.insert(suburb_index + 1, 'district')

# Reorder the DataFrame according to the new column order
df = df[columns]

In [177]:
df.to_parquet(os.path.join(parent_dir, 'DataAnalysis/Data/apartmentsBerlinDataCleaned.parquet'))
df.to_csv(os.path.join(parent_dir, 'DataAnalysis/Data/apartmentsBerlinDataCleaned.csv'))

In [178]:
print(f'The cleaned dataset has the length of {len(df)} and is saved as a parquet file.')

The cleaned dataset has the length of 8116 and is saved as a parquet file.
